In [8]:
!pip install gradio
!pip install gradio_client
!pip install whisperx
!pip install pydub

## Menu
##script_writing.py

mscript_input = "sex"
mscript_music_input = "sex"
final_video_output = "final_video_output.mp4"

Here’s a consistent and well-structured format where each script name aligns with the key variables:

**script_writing.py**

input_text (user_input)
script_output (scripttxt_output)
script_text (text)
audio_gen.py

script_audio_input (scripttxt_input)
audio_file (audio_output)
audio_format (mp3)
music_gen.py

script_music_input (scripttxt_input)
music_file (music_output)
music_format (mp3)
time_stamp_code.py

input_audio (audio_input)
timestamp_csv (tscsv_output)
csv_format (csv)
common_words_remover.py

raw_csv (csv_input)
filtered_csv (commoncsv_output)
csv_data (csv)
giphy_download.py

input_keywords (commoncsv_input)
downloaded_gif (gif_output)
gif_format (gif)
gif_crop_concat.py

raw_gif (gif_input)
gif_keywords (commoncsv_input)
final_video (concatenate_cropped_output)
video_format (mp4)
video_finalizer.py

video_input (concatenate_cropped_input)
audio_track (audio_input)
timestamps (tscsv_input)
background_music (music_input)
output_video (final_video_output)
output_format (mp4)

### script_writing.py

In [9]:
import csv
import re
from datetime import datetime
from gradio_client import Client

# Initialize the client with the correct Hugging Face Space
client = Client("Abu1998/Meme_finder")

# Define the system message and input sentence
system_message = """Task: Act as a YouTube Shorts content writer.

Objective: Create engaging, catchy, and trendy scripts for YouTube Shorts videos that are brief, attention-grabbing, and optimized for viral potential.

Guidelines:

Each script should be 15-30 seconds long.
Use a hook in the first few seconds to capture viewers' attention.
Ensure the content is aligned with trending topics, challenges, or popular culture.
Incorporate humor, relatable scenarios, or strong emotions to resonate with the audience.
End with a clear call-to-action (CTA) like “Follow for more!” or a cliffhanger.
Example Flow:

User Input: “Write a script about the Monday blues.”
AI Output:
Script: "POV: It’s Monday morning, and you’re already done with the week. [Clip shows someone groggily hitting the snooze button, dragging themselves out of bed]. But wait… there’s coffee. And suddenly, everything’s okay! ☕✨ [Cut to a quick burst of energy with upbeat music]. If you’re just surviving till the weekend, hit that follow button for more relatable vibes!"
"""

# Define the user input (the sentence for which you want to find the main keyword)
user_input = mscript_input

# Make the API call with the specified parameters
result = client.predict(
    message=user_input,
    system_message=system_message,
    max_tokens=512,
    temperature=0.7,
    top_p=0.95,
    api_name="/chat"
)

# Extract the script from the result
script = result.strip()

# Function to split script into words
def split_into_words(script_text):
    words = re.findall(r'\w+', script_text)  # Find all words
    return words

# Convert the script to a list of words
words = split_into_words(script)


# Define the file names with timestamp
csv_file = f'updates.csv'
txt_file = f'script_output.txt'

# Save to CSV
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Content', 'Word'])  # Headers
    for word in words:
        writer.writerow([user_input, word])  # Write each word as a separate row

print(f"Script generated, split into words, and saved to {csv_file}.")

# Save to TXT
with open(txt_file, mode='w', encoding='utf-8') as file:
    file.write(script)

print(f"Script saved to {txt_file}.")


Loaded as API: https://abu1998-meme-finder.hf.space ✔
Script generated, split into words, and saved to updates.csv.
Script saved to script_output.txt.


### audio_gen.py

In [10]:
# Install the gradio_client library


from gradio_client import Client
from google.colab import files
import shutil

# Initialize the client with the correct Hugging Face Space
client = Client("innoai/Edge-TTS-Text-to-Speech")

# Upload the script file
file_path = "/content/script_output.txt"

# Read the content from the uploaded script file
with open(file_path, 'r', encoding='utf-8') as file:
    text_input = file.read().strip()  # Read and strip any extra whitespace

# Make the API call with the file content as input
result = client.predict(
    text=text_input,
    voice="en-US-AvaMultilingualNeural - en-US (Female)",  # You can change the voice as needed
    rate=0,  # You can adjust the speech rate if needed
    pitch=9,  # You can adjust the pitch if needed
    api_name="/predict"
)

# Check the result type and content
print(result)

# Extract the local file path from the result
audio_file_path = result[0]  # Assuming the audio file path is the first element

# Define the output file name and path
output_file_path = "/content/audio_output.mp3"

# Copy the file to the desired location
shutil.copy(audio_file_path, output_file_path)

# Provide download link for the generated audio file
#files.download(output_file_path)


Loaded as API: https://innoai-edge-tts-text-to-speech.hf.space ✔
('/tmp/gradio/2540ecde4522c67543d51673f0e485f0bf750e20c33665e53045d2a68fa8dca2/tmpt7apw2_x.mp3', None)


'/content/audio_output.mp3'

###Music Gen

In [11]:
#from gradio_client import Client
#import requests

# Initialize the client with the correct Hugging Face Space URL
# Make sure the URL is correct and points to a valid Gradio app.
#client = Client("https://huggingface.co/spaces/facebook/MusicGen")  # Updated URL

# Define the input for the API
#input_text = "youtube shorts background music, beatbox, loop"

# Query the API (fn_index=2 is used based on the provided information)
#result = client.predict(
#    input_text,  # str input in 'parameter_16' Dataset component
#    fn_index=2
#)

# Check the result type
#print(result)

# Extract the URL from the result
# Assuming result is a tuple where the audio URL is the second element
#audio_url = result[1]

# Define the output file path
#output_file_path = "/content/tmp1mbn3d3s.mp4"

# Download the audio content from the URL
#response = requests.get(audio_url)
#audio_content = response.content

# Save the audio output to the specified path
#with open(output_file_path, 'wb') as f:
#    f.write(audio_content)

# Provide download link for the generated audio file
#from google.colab import files
#files.download(output_file_path)

### Time Stamp

In [22]:
import whisperx
import torch
import pandas as pd

# Initialize the WhisperX model
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float32" if device == "cpu" else "float16"
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

def transcribe_and_align(audio_file):
    # Load audio
    audio = whisperx.load_audio(audio_file)
    print("Audio loaded successfully.")

    # Transcribe
    result = model.transcribe(audio, batch_size=16)
    print("Transcription result:", result)

    # Align transcription
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=True)
    print("Alignment result:", result)

    # Process segments to get word-level timestamps
    word_segments = []
    for segment in result["segments"]:
        for word_info in segment.get("words", []):  # Ensure 'words' is used
            if "word" in word_info and "start" in word_info and "end" in word_info:
                word_segments.append({
                    "word": word_info["word"],
                    "start": word_info["start"],
                    "end": word_info["end"],
                    "duration": word_info["end"] - word_info["start"]
                })

    # Debug: Print word segments to check if they are being populated
    print("Word segments:", word_segments)

    # Convert the word segments to a DataFrame
    df = pd.DataFrame(word_segments)

    # Save the result to a CSV file
    output_file = "/content/transcription_with_word_timestamps.csv"  # Ensure correct file path
    df.to_csv(output_file, index=False)

    return output_file

# Provide the path to your audio file
audio_file_path = "/content/audio_output.mp3"
# Transcribe and align the audio file
output_file = transcribe_and_align(audio_file_path)

# Print the path to the output file
print(f"Word-level transcription with timestamps saved to: {output_file}")


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.1+cu121. Bad things might happen unless you revert torch to 1.x.
Audio loaded successfully.
Detected language: en (1.00) in first 30s of audio...
Transcription result: {'segments': [{'text': ' Title, The Benefits of Self Love.', 'start': 0.111, 'end': 2.551}, {'text': " Embracing Your Sexy Side. Script. Upbeat and energetic music plays as the video begins. Narrator. Hey there, gorgeous. Are you feeling confident and sexy today? If not, it's time to start loving yourself from the inside out. Self-love is the key to unlocking your true potential, and today we're going to show you how to embrace your sexy side. Cut to a close-up shot of a person applying lipstick in the mirror. Narrator. Start by looking i

### Duplicate of Time Stamp

In [21]:
import whisperx
import torch
import pandas as pd

# Initialize the WhisperX model
device = "cuda" if torch.cuda.is_available() else "cpu"
compute_type = "float32" if device == "cpu" else "float16"
model = whisperx.load_model("large-v2", device, compute_type=compute_type)

def transcribe_and_align(audio_file):
    # Load audio
    audio = whisperx.load_audio(audio_file)
    print("Audio loaded successfully.")

    # Transcribe
    result = model.transcribe(audio, batch_size=16,language='en')
    print("Transcription result:", result)

    # Align transcription
    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=True)
    print("Alignment result:", result)

    # Process segments to get word-level timestamps
    word_segments = []
    for segment in result["segments"]:
        for word_info in segment.get("words", []):  # Ensure 'words' is used
            if "word" in word_info and "start" in word_info and "end" in word_info:
                word_segments.append({
                    "word": word_info["word"],
                    "start": word_info["start"],
                    "end": word_info["end"],
                    "duration": word_info["end"] - word_info["start"]
                })

    # Debug: Print word segments to check if they are being populated
    print("Word segments:", word_segments)

    # Convert the word segments to a DataFrame
    df = pd.DataFrame(word_segments)

    # Save the result to a CSV file
    output_file = "/content/transcription_with_word_timestamps.csv"  # Ensure correct file path
    df.to_csv(output_file, index=False)

    return output_file

# Provide the path to your audio file
audio_file_path = "/content/audio_output.mp3"
# Transcribe and align the audio file
output_file = transcribe_and_align(audio_file_path)

# Print the path to the output file
print(f"Word-level transcription with timestamps saved to: {output_file}")


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.1+cu121. Bad things might happen unless you revert torch to 1.x.
Audio loaded successfully.
Transcription result: {'segments': [{'text': ' Title, The Benefits of Self Love.', 'start': 0.111, 'end': 2.551}, {'text': " Embracing Your Sexy Side. Script. Upbeat and energetic music plays as the video begins. Narrator. Hey there, gorgeous. Are you feeling confident and sexy today? If not, it's time to start loving yourself from the inside out. Self-love is the key to unlocking your true potential, and today we're going to show you how to embrace your sexy side. Cut to a close-up shot of a person applying lipstick in the mirror. Narrator. Start by looking in the mirror", 'start': 2.773, 'end': 32.756}, {'text'

###Khmer

### common_words_remover

In [23]:
# prompt: write a code to drop these common words from output_file word column , COMMON_WORDS = {"the", "and", "is", "in", "to", "of", "a", "with", "for", "on", "it", "as", "at", "by", "an","this", "that", "which", "or", "be", "are", "was", "were", "has", "have", "had", "why", "such","here", "some", "so", "easy"}

import pandas as pd
def drop_common_words(input_file, output_file, common_words):
  """
  Drops rows containing common words in the 'word' column and saves the result to a new CSV file.

  Args:
    input_file (str): The path to the input CSV file.
    output_file (str): The path to the output CSV file.
    common_words (set): A set of common words to be removed.
  """
  df = pd.read_csv(input_file)
  df['word'] = df['word'].str.lower()  # Convert words to lowercase for comparison
  df = df[~df['word'].isin(common_words)]  # Filter out rows with common words
  df.to_csv(output_file, index=False)

# Set of common words to drop
COMMON_WORDS = {"the", "and", "is", "in", "to", "of", "a", "with", "for", "on", "it", "as", "at", "by", "an","this", "that", "which", "or", "be", "are", "was", "were", "has", "have", "had", "why", "such","here", "some", "so", "easy"}

# Input and output file paths
input_file = "/content/transcription_with_word_timestamps.csv"
output_file = "/content/filtered_transcription.csv"

# Call the function to drop common words
drop_common_words(input_file, output_file, COMMON_WORDS)

print(f"Rows with common words dropped and saved to {output_file}")


Rows with common words dropped and saved to /content/filtered_transcription.csv


### common_words_remover 2nd step

In [25]:
import pandas as pd
from pydub import AudioSegment

def update_dataframe_with_audio_duration(csv_file, audio_file):
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    # Calculate the total duration of the audio
    audio = AudioSegment.from_file(audio_file)
    total_duration = audio.duration_seconds

    # Drop existing 'end' and 'duration' columns
    df = df.drop(columns=['end', 'duration'], errors='ignore')

    # Create a new 'end' column with the next 'start' value
    df['end'] = df['start'].shift(-1)

    # The first row should start with 0.01
    df.loc[0, 'start'] = 0.01

    # The last row's 'end' should be the total audio duration
    df.loc[df.index[-1], 'end'] = total_duration

    # Create a new 'duration' column based on the difference between 'start' and 'end'
    df['duration'] = df['end'] - df['start']
    df = df[df['duration'] >= 0.5]  # Filter out rows with duration less than 0.5 seconds ----------------------------------
    # Save the updated DataFrame back to CSV, extracting filename and prepending 'updated_'
    updated_csv_file = 'updated_' + csv_file.split('/')[-1] # Extract filename and prepend 'updated_'
    df.to_csv(updated_csv_file, index=False)

    print(f"Updated DataFrame saved to: {updated_csv_file}")
    return updated_csv_file

# Example usage
csv_file = '/content/filtered_transcription.csv'
audio_file = '/content/audio_output.mp3'
update_dataframe_with_audio_duration(csv_file, audio_file)

Updated DataFrame saved to: updated_filtered_transcription.csv


'updated_filtered_transcription.csv'

### Common_words_remover 2nd step update

### **Giphy Gif Download**

In [26]:
# prompt: write a code for "/content/dropped_2024-08-21_18-58-34.csv" to use Word column search in giphy api (API_KEY = "KzPlVn6nz6czmjWpPEy6reL52r1H5gs7") search and download in /content/memes this folder name as the word name

import requests
import csv
import os

# Giphy API details
API_KEY = "KzPlVn6nz6czmjWpPEy6reL52r1H5gs7"
SEARCH_URL = "https://api.giphy.com/v1/gifs/search"

# CSV and download directory
CSV_FILE = "/content/updated_filtered_transcription.csv"
DOWNLOAD_DIR = '/content/memes2'

# Create download directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_giphy_gif(search_term, filename):
  """Downloads a GIF from Giphy based on the search term."""
  params = {
      'api_key': API_KEY,
      'q': search_term,
      'limit': 1
  }
  response = requests.get(SEARCH_URL, params=params)
  data = response.json()

  if data['data']:
    gif_url = data['data'][0]['images']['original']['url']
    gif_response = requests.get(gif_url)

    with open(os.path.join(DOWNLOAD_DIR, filename), 'wb') as f:
      f.write(gif_response.content)
    print(f"Downloaded GIF for '{search_term}' as '{filename}'")
  else:
    print(f"No GIF found for '{search_term}'")

# Process the CSV file
with open(CSV_FILE, 'r', encoding='utf-8') as file:
  reader = csv.DictReader(file)
  for row in reader:
    word = row['word']
    filename = f"{word}.gif"
    download_giphy_gif((word,"Funny"), filename)


Downloaded GIF for '('title,', 'Funny')' as 'title,.gif'
Downloaded GIF for '('benefits', 'Funny')' as 'benefits.gif'
Downloaded GIF for '('love.', 'Funny')' as 'love..gif'
Downloaded GIF for '('embracing', 'Funny')' as 'embracing.gif'
Downloaded GIF for '('side.', 'Funny')' as 'side..gif'
Downloaded GIF for '('script.', 'Funny')' as 'script..gif'
Downloaded GIF for '('upbeat', 'Funny')' as 'upbeat.gif'
Downloaded GIF for '('plays', 'Funny')' as 'plays.gif'
Downloaded GIF for '('begins.', 'Funny')' as 'begins..gif'
Downloaded GIF for '('narrator.', 'Funny')' as 'narrator..gif'
Downloaded GIF for '('there,', 'Funny')' as 'there,.gif'
Downloaded GIF for '('gorgeous.', 'Funny')' as 'gorgeous..gif'
Downloaded GIF for '('confident', 'Funny')' as 'confident.gif'
Downloaded GIF for '('today?', 'Funny')' as 'today?.gif'
Downloaded GIF for '('not,', 'Funny')' as 'not,.gif'
Downloaded GIF for '('inside', 'Funny')' as 'inside.gif'
Downloaded GIF for '('out.', 'Funny')' as 'out..gif'
Downloaded GI

### Updated Download gif

In [ ]:
"""import requests
import csv
import os

# Giphy API details
API_KEY = "KzPlVn6nz6czmjWpPEy6reL52r1H5gs7"
SEARCH_URL = "https://api.giphy.com/v1/gifs/search"

# CSV and download directory
CSV_FILE = "/content/updated_filtered_transcription.csv"
DOWNLOAD_DIR = '/content/memes2'

# Create download directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_giphy_gif(search_term, index, offset=0):
    Downloads a GIF from Giphy based on the search term.
    params = {
        'api_key': API_KEY,
        'q': search_term,
        'limit': 1,
        'offset': offset
    }
    response = requests.get(SEARCH_URL, params=params)
    data = response.json()

    if data['data']:
        gif_url = data['data'][0]['images']['original']['url']
        gif_response = requests.get(gif_url)

        filename = f"{index}.gif"
        filepath = os.path.join(DOWNLOAD_DIR, filename)

        with open(filepath, 'wb') as f:
            f.write(gif_response.content)
        print(f"Downloaded GIF for '{search_term}' as '{filename}'")
        return gif_url
    else:
        print(f"No GIF found for '{search_term}'")
        return None

# Process the CSV file
with open(CSV_FILE, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    word_counts = {}
    for index, row in enumerate(reader):
        word = row['word']
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

        gif_url = download_giphy_gif(word, index, offset=word_counts[word] - 1)
        if gif_url:
            row['link'] = gif_url

# Write the updated CSV file
with open(CSV_FILE, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['word', 'link'])
    writer.writeheader()
    writer.writerows([{'word': word, 'link': gif_url} for word, gif_url in [(row['word'], row['link']) for row in reader]])"""

In [ ]:
# prompt: write a code to save /content/updated_filtered_transcription.csv with index
"""
import pandas as pd
df = pd.read_csv('/content/updated_filtered_transcription.csv')
df.to_csv('/content/updated_filtered_transcription.csv', index=True)
"""

In [ ]:
"""import requests
import csv
import os

# Giphy API details
API_KEY = "KzPlVn6nz6czmjWpPEy6reL52r1H5gs7"
SEARCH_URL = "https://api.giphy.com/v1/gifs/search"

# CSV and download directory
CSV_FILE = "/content/updated_filtered_transcription.csv"
DOWNLOAD_DIR = '/content/memes3'

# Create download directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_giphy_gif(search_term, index):
    """#Downloads a GIF from Giphy based on the search term.
    """
    params = {
        'api_key': API_KEY,
        'q': search_term,
        'limit': 1
    }
    response = requests.get(SEARCH_URL, params=params)
    data = response.json()

    if data['data']:
        gif_url = data['data'][0]['images']['original']['url']
        gif_response = requests.get(gif_url)

        filename = f"{index}.gif"
        with open(os.path.join(DOWNLOAD_DIR, filename), 'wb') as f:
            f.write(gif_response.content)
        print(f"Downloaded GIF for '{search_term}' as '{filename}'")
    else:
        print(f"No GIF found for '{search_term}'")

# Process the CSV file
with open(CSV_FILE, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for index, row in enumerate(reader):
        word = row['word']
        download_giphy_gif(word, index)"""

In [ ]:
"""import requests
import csv
import os

# Giphy API details
API_KEY = "KzPlVn6nz6czmjWpPEy6reL52r1H5gs7"
SEARCH_URL = "https://api.giphy.com/v1/gifs/search"

# CSV and download directory
CSV_FILE = "/content/updated_filtered_transcription.csv"
DOWNLOAD_DIR = '/content/memes3'

# Create download directory if it doesn't exist
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

def download_giphy_gif(search_term, index):
   Downloads a GIF from Giphy based on the search term.
    params = {
        'api_key': API_KEY,
        'q': search_term,
        'limit': 1
    }
    response = requests.get(SEARCH_URL, params=params)
    data = response.json()

    if data['data']:
        gif_url = data['data'][0]['images']['original']['url']
        gif_response = requests.get(gif_url)

        filename = f"{index}.gif"
        filepath = os.path.join(DOWNLOAD_DIR, filename)

        # Check if the file already exists
        if not os.path.exists(filepath):
            with open(filepath, 'wb') as f:
                f.write(gif_response.content)
            print(f"Downloaded GIF for '{search_term}' as '{filename}'")
        else:
            print(f"GIF for '{search_term}' already exists")

        return gif_url
    else:
        print(f"No GIF found for '{search_term}'")
        return None

# Process the CSV file
with open(CSV_FILE, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    rows = list(reader)  # Convert reader to a list to ensure it's fully read

# Check if any rows were read
if rows:
    # Add a "link" column if it doesn't exist
    if 'link' not in rows[0].keys():
        for row in rows:
            row['link'] = ''

    # Download GIFs and update the "link" column
    for index, row in enumerate(rows):
        word = row['word']
        gif_url = download_giphy_gif(word, index)
        if gif_url:
            row['link'] = gif_url

    # Write the updated CSV file
    with open(CSV_FILE, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=rows[0].keys())
        writer.writeheader()
        writer.writerows(rows)
else:
    print("The CSV file is empty.")"""

In [ ]:
"""import os
import pandas as pd

# Read the CSV file
df = pd.read_csv('/content/updated_filtered_transcription.csv')

# Get the list of words from the DataFrame
words = df['word'].tolist()

# Get the list of GIF files in the directory
gif_files = [f for f in os.listdir('/content/memes2') if f.endswith('.gif')]

# Ensure we have enough words for the GIF files
if len(words) < len(gif_files):
    raise ValueError("Not enough words in the CSV file to rename all GIFs.")

# Iterate through the GIF files and rename them
for i, gif_file in enumerate(gif_files):
    try:
        # Generate new filename based on index from the CSV
        new_filename = f"{i+1}.gif"  # Index starts from 1

        # Construct the full paths for the old and new filenames
        old_path = os.path.join('/content/memes2', gif_file)
        new_path = os.path.join('/content/memes2', new_filename)

        # Rename the file
        os.rename(old_path, new_path)
        print(f"Renamed '{gif_file}' to '{new_filename}'")
    except Exception as e:
        print(f"Error processing file '{gif_file}': {e}")
"""

### Concate Gif (incomplete)

In [ ]:
# prompt: drop Unnamed: 0 and save the file

#import pandas as pd
#df = pd.read_csv('/content/updated_filtered_transcription.csv')
#df = df.drop(columns=['Unnamed: 0'])
#df.to_csv('/content/updated_filtered_transcription.csv', index=False)
df.head()

In [27]:
import moviepy.editor as mpe
import os
import csv

# CSV and download directory paths
CSV_FILE = '/content/updated_filtered_transcription.csv'
DOWNLOAD_DIR = '/content/memes2'
OUTPUT_VIDEO = 'updated_concatenated_memes.mp4'

# Get the GIF order and durations from the CSV file
gif_order = []
durations = {}
with open(CSV_FILE, 'r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    for row in reader:
        gif_filename = row['word'] + '.gif'
        duration = float(row['duration'])  # Ensure this matches the column name in your CSV
        gif_order.append(gif_filename)
        durations[gif_filename] = duration

# Load, crop, and concatenate GIFs
clips = []
for gif_filename in gif_order:
    gif_path = os.path.join(DOWNLOAD_DIR, gif_filename)
    if os.path.exists(gif_path):
        clip = mpe.VideoFileClip(gif_path).resize(height=480)  # Resize to the same height
        clip = clip.set_fps(24)  # Match the frame rate for consistency

        # Crop each GIF to the specified duration from the new CSV
        max_duration = durations.get(gif_filename, clip.duration)  # Use the duration from the CSV or the full clip duration if not found
        if clip.duration > max_duration:
            clip = clip.subclip(0, max_duration)  # Keep up to the specified duration

        clips.append(clip)
    else:
        print(f"Warning: GIF not found: {gif_filename}")

# Concatenate and save the video
if clips:
    final_clip = mpe.concatenate_videoclips(clips, method="compose")
    final_clip.write_videofile(OUTPUT_VIDEO, fps=24)  # Set fps to match the GIFs
    print(f"Concatenated video saved as {OUTPUT_VIDEO}")
else:
    print("No GIFs found to concatenate.")


Moviepy - Building video updated_concatenated_memes.mp4.
Moviepy - Writing video updated_concatenated_memes.mp4



Moviepy - Done !
Moviepy - video ready updated_concatenated_memes.mp4
Concatenated video saved as updated_concatenated_memes.mp4


### concate_audio_gif_music

In [ ]:
import moviepy.editor as mpe
import os

# File paths
video_file = '/content/updated_concatenated_memes.mp4'
music_file = '/content/tmp1mbn3d3s.mp4'
audio_file = '/content/audio_output.mp3'
output_file = '/content/final_output.mp4'

# Load the video, music, and audio files
video_clip = mpe.VideoFileClip(video_file)
music_clip = mpe.VideoFileClip(music_file)
audio_clip = mpe.AudioFileClip(audio_file)

# Duration of the video
video_duration = video_clip.duration

# Ensure the music duration matches the video duration
if music_clip.duration < video_duration:
    # Repeat the music to match the video duration
    n_repeats = int(video_duration // music_clip.duration) + 1
    music_clip = mpe.concatenate_videoclips([music_clip] * n_repeats).subclip(0, video_duration)
elif music_clip.duration > video_duration:
    music_clip = music_clip.subclip(0, video_duration)

# Adjust music volume to 50% and keep audio volume at 100%
music_clip = music_clip.volumex(0.3)  # Reduce music volume to 50%

# Ensure the audio duration matches the video duration
if audio_clip.duration < video_duration:
    # Repeat the audio to match the video duration
    n_repeats = int(video_duration // audio_clip.duration) + 1
    audio_clip = mpe.concatenate_audioclips([audio_clip] * n_repeats).subclip(0, video_duration)
elif audio_clip.duration > video_duration:
    audio_clip = audio_clip.subclip(0, video_duration)

# Set the audio of the video clip to the adjusted audio
video_clip = video_clip.set_audio(audio_clip)

# Write the final output video with the adjusted music and audio
final_clip = video_clip.set_audio(music_clip.audio)
final_clip.write_videofile(output_file, codec='libx264', audio_codec='aac')

print(f"Final video saved as {output_file}")


Moviepy - Building video /content/final_output.mp4.
MoviePy - Writing audio in final_outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /content/final_output.mp4



t:  88%|████████▊ | 1801/2056 [00:28<00:05, 48.79it/s, now=None]

In [ ]:
import moviepy.editor as mpe
import os

# File paths
video_file = '/content/updated_concatenated_memes.mp4'
music_file = '/content/tmp1mbn3d3s.mp4'
audio_file = '/content/tmp3dbnak4t.mp3'
output_file = '/content/final_output.mp4'

# Load the video, music, and audio files
video_clip = mpe.VideoFileClip(video_file)
music_clip = mpe.VideoFileClip(music_file)
audio_clip = mpe.AudioFileClip(audio_file)

# Duration of the video
video_duration = video_clip.duration

# Ensure the music duration matches the video duration
if music_clip.duration < video_duration:
    # Repeat the music to match the video duration
    n_repeats = int(video_duration // music_clip.duration) + 1
    music_clip = mpe.concatenate_videoclips([music_clip] * n_repeats).subclip(0, video_duration)
elif music_clip.duration > video_duration:
    music_clip = music_clip.subclip(0, video_duration)

# Ensure the audio duration matches the video duration
if audio_clip.duration < video_duration:
    # Repeat the audio to match the video duration
    n_repeats = int(video_duration // audio_clip.duration) + 1
    audio_clip = mpe.concatenate_audioclips([audio_clip] * n_repeats).subclip(0, video_duration)
elif audio_clip.duration > video_duration:
    audio_clip = audio_clip.subclip(0, video_duration)

# Adjust music volume to 50% and keep audio volume at 100%
music_clip = music_clip.volumex(0.3)  # Reduce music volume to 50%

# Set the audio of the video clip to the adjusted audio
video_clip = video_clip.set_audio(audio_clip)

# Combine the video with adjusted music
final_audio = mpe.CompositeAudioClip([music_clip.audio, audio_clip])
final_clip = video_clip.set_audio(final_audio)

# Write the final output video
final_clip.write_videofile(output_file, codec='libx264', audio_codec='aac')

print(f"Final video saved as {output_file}")
